# 🏆 Arbitrium Framework - Interactive Browser Demo

## Tournament-Based AI Decision Synthesis - Live in Your Browser!

Welcome! This interactive notebook lets you run **real** Arbitrium tournaments with **actual AI models** right here in your browser.

**What you'll experience:**
- 🤖 Real AI models (GPT-4, Claude, Gemini) competing on your question
- 🔄 Live tournament execution with real API calls
- ⚔️ Competitive elimination rounds
- 🧠 Knowledge Bank preserving insights from eliminated models
- 🥇 Champion solution synthesizing the best ideas
- 💰 Real cost tracking and metrics

**Requirements:**
- Your own API keys (OpenAI, Anthropic, or Google)
- ~$0.50-2.00 per tournament (depending on models used)
- 5-10 minutes runtime

---

## Step 1: Install Arbitrium Framework

First, let's install the framework. In Pyodide/JupyterLite environments, use `micropip`:

In [ ]:
# Installation - works for Colab, JupyterLite, and local Jupyter

# Try to import first (in case already installed)
try:
    import arbitrium
    print(f"✅ Arbitrium already installed (v{arbitrium.__version__})")
except ImportError:
    print("📦 Installing Arbitrium Framework...\n")
    
    # Try PyPI first (once published)
    try:
        # For Pyodide/JupyterLite
        try:
            import micropip
            await micropip.install('arbitrium-framework')
            print("✅ Installed via micropip (Pyodide environment)")
        except:
            # For regular Python/Colab
            !{sys.executable} -m pip install -q arbitrium
            print("✅ Installed via pip")
        
        import arbitrium
        print(f"\n🎉 Arbitrium Framework v{arbitrium.__version__} ready!")
        
    except Exception:
        # Fallback: Install from GitHub (if not yet on PyPI)
        print("⚠️  Not found on PyPI, installing from GitHub...")
        !{sys.executable} -m pip install -q git+https://github.com/arbitrium-framework/arbitrium.git
        import arbitrium
        print(f"✅ Installed from GitHub (v{arbitrium.__version__})\n")

# Import and verify
import arbitrium
print("🎉 Ready to run tournaments!")

## Step 2: Configure Your API Keys

**Security Note:** Your API keys stay in your browser session and are never sent anywhere except directly to the AI providers.

Choose which models you want to use:

In [ ]:
import os
import getpass

print("🔐 API Key Configuration")
print("=" * 60)
print("\nEnter API keys for the models you want to use.")
print("Press Enter to skip a provider.\n")

# OpenAI (GPT-4)
if not os.getenv('OPENAI_API_KEY'):
    openai_key = getpass.getpass("OpenAI API Key (for GPT-4): ")
    if openai_key.strip():
        os.environ['OPENAI_API_KEY'] = openai_key.strip()
        print("  ✅ OpenAI configured")
    else:
        print("  ⏭️  OpenAI skipped")
else:
    print("  ✅ OpenAI already configured")

# Anthropic (Claude)
if not os.getenv('ANTHROPIC_API_KEY'):
    anthropic_key = getpass.getpass("Anthropic API Key (for Claude): ")
    if anthropic_key.strip():
        os.environ['ANTHROPIC_API_KEY'] = anthropic_key.strip()
        print("  ✅ Anthropic configured")
    else:
        print("  ⏭️  Anthropic skipped")
else:
    print("  ✅ Anthropic already configured")

# Google (Gemini)
if not os.getenv('GOOGLE_API_KEY'):
    google_key = getpass.getpass("Google API Key (for Gemini): ")
    if google_key.strip():
        os.environ['GOOGLE_API_KEY'] = google_key.strip()
        print("  ✅ Google configured")
    else:
        print("  ⏭️  Google skipped")
else:
    print("  ✅ Google already configured")

# XAI (Grok) - optional
if not os.getenv('XAI_API_KEY'):
    xai_key = getpass.getpass("XAI API Key (for Grok, optional): ")
    if xai_key.strip():
        os.environ['XAI_API_KEY'] = xai_key.strip()
        print("  ✅ XAI configured")
    else:
        print("  ⏭️  XAI skipped")
else:
    print("  ✅ XAI already configured")

print("\n" + "=" * 60)
print("✅ Configuration complete!\n")

## Step 3: Create Tournament Configuration

Let's build a configuration dynamically based on which API keys you provided:

In [ ]:
import tempfile
from pathlib import Path

# Build model configuration based on available API keys
models_config = {}

if os.getenv('OPENAI_API_KEY'):
    models_config['gpt4'] = {
        'provider': 'openai',
        'model_name': 'gpt-4-turbo',
        'display_name': 'GPT-4 Turbo'
    }

if os.getenv('ANTHROPIC_API_KEY'):
    models_config['claude'] = {
        'provider': 'anthropic',
        'model_name': 'claude-3-5-sonnet-20241022',
        'display_name': 'Claude 3.5 Sonnet'
    }

if os.getenv('GOOGLE_API_KEY'):
    models_config['gemini'] = {
        'provider': 'gemini',
        'model_name': 'gemini-1.5-pro',
        'display_name': 'Gemini 1.5 Pro'
    }

if os.getenv('XAI_API_KEY'):
    models_config['grok'] = {
        'provider': 'xai',
        'model_name': 'grok-2',
        'display_name': 'Grok 2'
    }

# ⚠️ VALIDATION: Check for sufficient models
if not models_config:
    raise ValueError(
        "❌ No API keys configured! Please run Step 2 and provide at least one API key."
    )

if len(models_config) < 2:
    print("\n⚠️  WARNING: INSUFFICIENT MODELS FOR TOURNAMENT")
    print("=" * 60)
    print(f"\n❌ Found only {len(models_config)} active model(s):")
    for _key, model in models_config.items():
        print(f"   • {model['display_name']}")
    print("\n🏆 Arbitrium tournaments require at least 2 models to compete.")
    print("\n💡 To fix this:")
    print("   1. ⬆️  Scroll up to Step 2")
    print("   2. Re-run that cell and enter at least one more API key")
    print("   3. Then come back and re-run Steps 3-6")
    print("\n🔑 Need API keys? Get them here:")
    print("   • OpenAI:    https://platform.openai.com/api-keys")
    print("   • Anthropic: https://console.anthropic.com/settings/keys")
    print("   • Google:    https://aistudio.google.com/app/apikey")
    print("   • XAI:       https://console.x.ai/")
    print("\n" + "=" * 60 + "\n")
    raise ValueError(
        f"Tournament requires 2+ models, but only {len(models_config)} configured. "
        "Please add more API keys in Step 2."
    )

# Create temporary directory for outputs
temp_dir = tempfile.mkdtemp(prefix="arbitrium_demo_")

# Full configuration
config = {
    'models': models_config,
    'outputs_dir': temp_dir,
    'retry': {
        'max_attempts': 3,
        'initial_delay': 1,
        'max_delay': 10,
        'exponential_base': 2
    },
    'features': {
        'save_reports_to_disk': True,
        'deterministic_mode': False,
        'judge_model': None,  # Use peer review
        'knowledge_bank_model': 'leader',
        'llm_compression': False
    },
    'knowledge_bank': {
        'enabled': True,
        'similarity_threshold': 0.75,
        'max_insights_to_inject': 5
    }
}

print("🎯 Tournament Configuration")
print("=" * 60)
print(f"\n📊 Active Models ({len(models_config)}):")
for _key, model in models_config.items():
    print(f"  • {model['display_name']} ({model['model_name']})")

print(f"\n🧠 Knowledge Bank: {'Enabled' if config['knowledge_bank']['enabled'] else 'Disabled'}")
print(f"📁 Output Directory: {temp_dir}")
print("\n" + "=" * 60 + "\n")

## Step 4: Initialize Arbitrium Framework

Now let's initialize the framework and perform health checks on all models:

In [ ]:
from arbitrium import Arbitrium

print("🏁 Initializing Arbitrium Framework...\n")

# Initialize from settings
arbitrium = await Arbitrium.from_settings(
    settings=config,
    skip_secrets=True,  # We already set environment variables
    skip_health_check=False  # DO perform health checks
)

print("\n" + "=" * 60)
print("📊 Model Health Check Results")
print("=" * 60 + "\n")

print(f"✅ Healthy Models: {arbitrium.healthy_model_count}")
if arbitrium.healthy_model_count > 0:
    for model_key in arbitrium.healthy_models.keys():
        model = arbitrium.healthy_models[model_key]
        print(f"   • {model.full_display_name}")

if arbitrium.failed_model_count > 0:
    print(f"\n❌ Failed Models: {arbitrium.failed_model_count}")
    for model_key, error in arbitrium.failed_models.items():
        print(f"   • {model_key}: {error}")
    print("\n⚠️  Continuing with healthy models only.")

print("\n" + "=" * 60 + "\n")

if arbitrium.healthy_model_count < 2:
    raise ValueError(
        "❌ Need at least 2 healthy models to run a tournament! "
        "Please check your API keys and try again."
    )

print(f"🎉 Ready to run tournament with {arbitrium.healthy_model_count} models!\n")

## Step 5: Define Your Strategic Question

Choose a question for the tournament. This should be a high-stakes, complex decision where multiple perspectives add value.

**Examples:**
- Strategic business decisions
- Technical architecture choices
- Market analysis and positioning
- Product roadmap prioritization

**Edit the question below or use the example:**

In [ ]:
# Define your question here
question = """
Analyze the strategic decision: Should our SaaS startup focus on 
product-led growth (PLG) or sales-led growth (SLG) for our first year?

Context:
- Pre-revenue, team of 3 engineers
- B2B product (developer tools for API testing)
- $500k seed funding, 12-month runway
- Competitive landscape: established players (Postman, Insomnia)
- Target: 10,000 users or $500k ARR by end of year

Consider:
- Customer acquisition costs
- Time to first revenue
- Scalability with small team
- Market expectations in this category
- Risk vs. speed trade-offs
""".strip()

print("❓ Tournament Question")
print("=" * 60)
print(f"\n{question}\n")
print("=" * 60 + "\n")

## Step 6: Run the Tournament! 🏆

This will execute the full tournament:
1. **Phase 1**: Each model generates independent initial responses
2. **Phase 2**: Models improve based on peer feedback
3. **Phase 3**: Cross-evaluation and elimination
4. **Repeat**: Until one champion remains

**Expected runtime:** 5-10 minutes (depending on number of models)

**Expected cost:** $0.50-$2.00 (you'll see exact costs below)

In [ ]:
import time

print("\n" + "=" * 80)
print("🏆 ARBITRIUM TOURNAMENT - LIVE EXECUTION")
print("=" * 80 + "\n")

start_time = time.time()

# Run the tournament with REAL API calls
result, metrics = await arbitrium.run_tournament(question)

end_time = time.time()
duration = end_time - start_time

print("\n" + "=" * 80)
print("🏁 TOURNAMENT COMPLETE")
print("=" * 80 + "\n")

print(f"⏱️  Duration: {duration:.1f} seconds ({duration/60:.1f} minutes)")
print(f"💰 Total Cost: ${metrics['total_cost']:.4f}")
print(f"🥇 Champion: {metrics['champion_model']}")
print(f"🗑️  Eliminated: {len(metrics['eliminated_models'])} models")

if metrics['eliminated_models']:
    print("\n   Elimination order:")
    for i, model in enumerate(metrics['eliminated_models'], 1):
        print(f"   {i}. {model}")

print("\n💸 Cost Breakdown:")
for model, cost in sorted(metrics['cost_by_model'].items(), key=lambda x: x[1], reverse=True):
    pct = (cost / metrics['total_cost'] * 100) if metrics['total_cost'] > 0 else 0
    print(f"   {model:20} ${cost:7.4f} ({pct:.1f}%)")

print("\n" + "=" * 80 + "\n")

## Step 7: View Champion Solution

Here's the winning answer that synthesizes insights from all competing models:

In [ ]:
from IPython.display import Markdown, display

print("=" * 80)
print("📜 CHAMPION SOLUTION")
print("=" * 80 + "\n")

display(Markdown(result))

print("\n" + "=" * 80 + "\n")

## Step 8: Download Tournament Reports

Arbitrium Framework generates detailed reports you can download:

In [ ]:
import glob

print("📁 Tournament Reports")
print("=" * 60 + "\n")

# Find generated reports
report_files = glob.glob(f"{temp_dir}/arbitrium_*.json") + glob.glob(f"{temp_dir}/arbitrium_*.md")

if report_files:
    print(f"✅ Generated {len(report_files)} report files:\n")
    for filepath in sorted(report_files):
        filename = Path(filepath).name
        size_kb = Path(filepath).stat().st_size / 1024
        print(f"   📄 {filename} ({size_kb:.1f} KB)")
    
    print(f"\n📂 Location: {temp_dir}")
    print("\n💡 Tip: These files contain:")
    print("   • Complete tournament history (all responses, scores)")
    print("   • Provenance tracking (how ideas evolved)")
    print("   • Knowledge Bank insights")
    print("   • Cost breakdown by phase")
else:
    print("⚠️  No report files found (may be disabled in config)")

print("\n" + "=" * 60 + "\n")

## Step 9: Compare with Single Model (Optional)

Want to see how the tournament compares to just asking one model? Let's run the same question through a single model:

In [ ]:
# Choose first healthy model for comparison
comparison_model_key = next(iter(arbitrium.healthy_models.keys()))
comparison_model = arbitrium.healthy_models[comparison_model_key]

print(f"🤖 Running single model comparison: {comparison_model.full_display_name}\n")

single_response = await arbitrium.run_single_model(comparison_model_key, question)

print("\n" + "=" * 80)
print(f"📝 SINGLE MODEL RESPONSE ({comparison_model.full_display_name})")
print("=" * 80 + "\n")

display(Markdown(single_response.content))

print("\n" + "=" * 80)
print("⚖️  COMPARISON")
print("=" * 80 + "\n")

print(f"Tournament Champion: {metrics['champion_model']}")
print(f"Tournament Cost: ${metrics['total_cost']:.4f}")
print(f"Tournament Time: {duration:.1f}s")
print(f"\nSingle Model: {comparison_model.full_display_name}")
print(f"Single Model Cost: ${single_response.cost:.4f}")
print(f"\n💰 Cost Multiplier: {metrics['total_cost'] / single_response.cost:.1f}x")
print("\n💡 Analysis: Is the tournament answer worth the extra cost?")
print("   Read both answers above and decide!")

print("\n" + "=" * 80 + "\n")

## Step 10: Try Your Own Question!

Now that you've seen it work, modify the question in Step 5 and run Steps 6-9 again with your own strategic problem.

**Tips for good tournament questions:**
- Complex with multiple valid approaches
- High-stakes ($5,000+ impact)
- Benefits from diverse perspectives
- Has context and constraints
- No single "obviously correct" answer

**Poor tournament questions:**
- Simple factual queries
- Questions with objective right answers
- Very broad without constraints
- Low-stakes decisions

---

## Summary: What You Just Experienced

### 🎉 Congratulations!

You just ran a **real Arbitrium Framework tournament** with actual AI models:

1. ✅ **Multiple AI models competed** with real API calls
2. ✅ **Competitive pressure** drove higher quality responses
3. ✅ **Knowledge Bank preserved** insights from eliminated models
4. ✅ **Champion solution synthesized** the best ideas from all perspectives
5. ✅ **Full traceability** with downloadable reports
6. ✅ **Cost tracking** showed exactly what you spent

### 📊 Tournament vs. Single Model

**When Tournament Wins:**
- Multiple valid approaches to explore
- High stakes justify the cost
- Need defensible audit trail
- Synthesis of diverse viewpoints matters

**When Single Model is Fine:**
- Simple queries
- Time-sensitive decisions
- Low-stakes problems
- Budget constraints

### 🚀 Next Steps

**Use Arbitrium Locally:**
```bash
pip install arbitrium-framework
cp config.example.yml config.yml
# Edit config.yml with your API keys
arbitrium  # Run CLI
```

**Programmatic Usage:**
```python
from arbitrium import Arbitrium

arbitrium = await Arbitrium.from_config("config.yml")
result, metrics = await arbitrium.run_tournament("Your question")
```

**Learn More:**
- 📖 [Documentation](https://github.com/arbitrium-framework/arbitrium)
- 🎯 [More Examples](https://github.com/arbitrium-framework/arbitrium/tree/main/examples)
- 💬 [Discord Community](https://discord.gg/arbitrium)
- 🐛 [Report Issues](https://github.com/arbitrium-framework/arbitrium/issues)

### 💡 Key Takeaways

1. **Competitive synthesis works**: Tournament pressure + Knowledge Bank preservation = better outcomes
2. **Real cost tracking**: Always know what you're spending
3. **Full auditability**: Every decision is traceable
4. **Model-agnostic**: Use any combination of OpenAI, Anthropic, Google, etc.
5. **Right tool for the job**: Use tournaments for high-stakes decisions, single models for everything else

---

**Thank you for trying Arbitrium Framework!**

If this helped solve a real decision for you, consider:
- ⭐ Starring the [GitHub repo](https://github.com/arbitrium-framework/arbitrium)
- 📢 Sharing your results (with permission)
- 🤝 Contributing improvements

*Arbitrium Framework™ - Tournament-Based AI Decision Synthesis*

*MIT License | Not affiliated with Arbitrum or Arbitrium RAT*